In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from dataset_utils import read_annotations, FOLDS, FOLDS_ROOT, IMAGES_ROOT, EXTRACTED_IMAGES_ROOT
from plotting import plot_image_with_bounding_boxes, load_image_to_array

split_type = "TRAIN"
folds = FOLDS[split_type]

all_annotations_per_image = read_annotations(FOLDS_ROOT, folds)

In [ ]:
for key in list(all_annotations_per_image.keys())[:10]:

    bboxes = all_annotations_per_image[key]

    img_array = load_image_to_array(IMAGES_ROOT / (key + ".jpg"))
    plot_image_with_bounding_boxes(img_array, bboxes)

In [ ]:
all_bbox_widths = []
all_bbox_heights = []

for bboxes in all_annotations_per_image.values():
    for bbox in bboxes:
        all_bbox_heights.append(bbox[2] - bbox[0])
        all_bbox_widths.append(bbox[3] - bbox[1])

plt.hist(all_bbox_widths)
plt.hist(all_bbox_heights)

print("Median width: ", np.quantile(all_bbox_widths, 0.5))
print("Median height: ", np.quantile(all_bbox_heights, 0.5))
print("Median aspect ratio: ", np.quantile(np.array(all_bbox_heights) / np.array(all_bbox_widths), 0.5))

Aspect ratio makes sense! Extract rectangles of dimension (80, 120) or reshape images to that size before exporting them!

Extract images:

In [ ]:
target_width = 80
target_height = 120
num_background_per_image = 2

(EXTRACTED_IMAGES_ROOT / split_type / "0").mkdir(parents=True, exist_ok=True)
(EXTRACTED_IMAGES_ROOT / split_type / "1").mkdir(parents=True, exist_ok=True)

rng = np.random.default_rng(seed=0)


def bb_intersection_over_union(boxA, boxB):
    # Function from https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou


def normalize_box(bbox, image_shape):
     h0, w0, h1, w1 = bbox

     return (max(h0, 0), max(w0, 0), min(h1, image_shape[0]), min(w1, image_shape[1]))


for image_path, annotations_on_image in list(all_annotations_per_image.items()):

    img = Image.open(IMAGES_ROOT / (image_path + ".jpg"))
    img_array = np.asarray(img)
    # plt.imshow(img_array)
    # plt.show()

    # Crop images of faces
    for count, bbox in enumerate(annotations_on_image):
        int_bbox = tuple(int(bbox_coord) for bbox_coord in bbox)
        int_bbox = normalize_box(int_bbox, img_array.shape)
        cropped_face_image = Image.fromarray(img_array[int_bbox[0]:int_bbox[2], int_bbox[1]:int_bbox[3]])
        cropped_face_image = cropped_face_image.resize((target_width, target_height), Image.LANCZOS)
        # plt.imshow(cropped_face_image)
        # plt.show()

        cropped_face_image.save(EXTRACTED_IMAGES_ROOT / split_type / "1" / (image_path.replace("/", "_") + f"_{count}.jpg"))

    # Crop background images
    bg_image_counter = 0
    while bg_image_counter < num_background_per_image:
        center_height = rng.integers(0, high=img_array.shape[0])
        center_width = rng.integers(0, high=img_array.shape[1])

        half_width = rng.integers(20, 60)
        half_height = int(half_width * target_height / target_width)
        
        # TODO: I think this computation of the bounding box is wrong!
        bg_box = (center_height - half_height, center_width - half_width, center_height + half_height, center_width + half_width)

        if (bg_box[0] < 0) or (bg_box[1] < 0) or (bg_box[2] > img_array.shape[0]) or (bg_box[3] > img_array.shape[1]):
            print("Unsuitable image")
            continue

        if any(bb_intersection_over_union(bg_box, annotation) > 0.1 for annotation in annotations_on_image):
             print("Too large overlap")
             continue

        cropped_bg_image = Image.fromarray(img_array[bg_box[0]:bg_box[2], bg_box[1]:bg_box[3]])
        cropped_bg_image = cropped_bg_image.resize((target_width, target_height), Image.LANCZOS)
        # plt.imshow(cropped_bg_image)
        # plt.show()

        cropped_bg_image.save(EXTRACTED_IMAGES_ROOT / split_type / "0" / (image_path.replace("/", "_") + f"_{bg_image_counter}.jpg"))
        bg_image_counter += 1
        print("Success")